# 🇮🇩 Liputan6 — Fine-tuning BERT2BERT (20% Sampling + ROUGE Save, Colab T4)

Model: `cahya/bert2bert-indonesian-summarization`  
Data: `BASE_DIR / Data / liputan6_clean_ready.csv`  
Outputs: `BASE_DIR / Outputs`


In [1]:
# 🚀 Mount + Path Config (consistent with Step-2)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/Tugas/Liputan6")  #@param {type:"string"}
DATA_PATH = BASE_DIR / "EDA_Outputs" / "liputan6_clean_ready.csv"
OUTPUT_DIR = BASE_DIR / "Outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print("Using DATA_PATH:", DATA_PATH)
print("Using OUTPUT_DIR:", OUTPUT_DIR)


Mounted at /content/drive
Using DATA_PATH: /content/drive/MyDrive/Tugas/Liputan6/EDA_Outputs/liputan6_clean_ready.csv
Using OUTPUT_DIR: /content/drive/MyDrive/Tugas/Liputan6/Outputs


In [2]:
!pip -q install --upgrade pip
!pip -q install transformers==4.44.2 datasets==2.21.0 sentencepiece accelerate evaluate rouge_score pandas numpy --quiet
!pip -q install torch --index-url https://download.pytorch.org/whl/cu121 --quiet
import torch, pandas as pd, numpy as np
print("PyTorch:", torch.__version__, "CUDA:", torch.cuda.is_available())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.
PyTorch: 2.8.0+cu126 CUDA: True


In [3]:
# ⚙️ Training Config
MODEL_NAME = "cahya/bert2bert-indonesian-summarization"  #@param {type:"string"}
TEXT_COL = "clean_article_text"                          #@param {type:"string"}
SUMMARY_COL = "clean_summary_text"                       #@param {type:"string"}
MAX_INPUT_LEN = 512                                       #@param {type:"integer"}
MAX_TARGET_LEN = 128                                      #@param {type:"integer"}
NUM_EPOCHS = 1                                            #@param {type:"integer"}
LR = 5e-5                                                 #@param {type:"number"}
TRAIN_BS = 2                                              #@param {type:"integer"}
EVAL_BS = 2                                               #@param {type:"integer"}
GRAD_ACCUM = 2                                            #@param {type:"integer"}
VAL_SPLIT = 0.05                                          #@param {type:"number"}
USE_FP16 = True                                           #@param {type:"boolean"}
SAMPLE_FRAC = 0.2                                         #@param {type:"number"}
SAVE_DIR = OUTPUT_DIR / "models" / "bert2bert_finetuned_20pct"
SAVE_DIR.mkdir(parents=True, exist_ok=True)
print("Model save dir:", SAVE_DIR)


Model save dir: /content/drive/MyDrive/Tugas/Liputan6/Outputs/models/bert2bert_finetuned_20pct


In [4]:
# 📥 Load data + use 20% sampling
df = pd.read_csv(DATA_PATH, low_memory=False)
assert TEXT_COL in df.columns, f"Missing TEXT_COL {TEXT_COL}. Found: {list(df.columns)}"
assert SUMMARY_COL in df.columns, f"Missing SUMMARY_COL {SUMMARY_COL}. Found: {list(df.columns)}"
orig_n = len(df)
df = df.sample(frac=SAMPLE_FRAC, random_state=42).reset_index(drop=True)
print(f"Using {len(df)} rows ({SAMPLE_FRAC*100:.0f}% of {orig_n})")
df[[TEXT_COL, SUMMARY_COL]].head(2)


Using 38602 rows (20% of 193011)


,clean_article_text,clean_summary_text
0,"jakarta: husein mutahar, rabu ( 9/6 ), pukul 1...",mutahar meninggal dunia pada usia 88 tahun set...
1,manchester city berhasil mempertahankan rekor ...,manchester city mempertahankan rekor tak terka...


In [5]:
# ✂️ Train/Validation split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=VAL_SPLIT, random_state=42)
print("Train:", len(train_df), "Val:", len(val_df))


Train: 36671 Val: 1931


In [6]:
# 🔧 Tokenizer & model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
print("Model loaded.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

Model loaded.


In [7]:
# 🧾 Dataset class
import torch
from torch.utils.data import Dataset

class SummDataset(Dataset):
    def __init__(self, df, text_col, summary_col, tokenizer, max_input_len, max_target_len):
        self.df = df.reset_index(drop=True)
        self.text_col = text_col
        self.summary_col = summary_col
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        src = str(row[self.text_col])
        tgt = str(row[self.summary_col])
        model_inputs = self.tokenizer(src, max_length=self.max_input_len, truncation=True, padding="max_length", return_tensors="pt")
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(tgt, max_length=self.max_target_len, truncation=True, padding="max_length", return_tensors="pt")
        model_inputs = {k: v.squeeze(0) for k, v in model_inputs.items()}
        model_inputs["labels"] = labels["input_ids"].squeeze(0)
        return model_inputs


In [8]:
# 🔨 Build datasets
train_ds = SummDataset(train_df, TEXT_COL, SUMMARY_COL, tokenizer, MAX_INPUT_LEN, MAX_TARGET_LEN)
val_ds = SummDataset(val_df, TEXT_COL, SUMMARY_COL, tokenizer, MAX_INPUT_LEN, MAX_TARGET_LEN)
len(train_ds), len(val_ds)


(36671, 1931)

In [9]:
# 🧪 ROUGE metric
import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    scores = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: float(v) for k, v in scores.items()}


In [ ]:
# ⚙️ Trainer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
args = Seq2SeqTrainingArguments(
    output_dir=str(SAVE_DIR),
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    per_device_eval_batch_size=EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    predict_with_generate=True,
    fp16=USE_FP16,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()
trainer.save_model(str(SAVE_DIR))
tokenizer.save_pretrained(str(SAVE_DIR))
print("Model saved to:", SAVE_DIR)


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gyeran-huray (gyeran-huray-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decode

Epoch,Training Loss,Validation Loss


In [ ]:
# 📏 Evaluate on validation set and save ROUGE metrics
metrics = trainer.evaluate()
print("ROUGE metrics:", metrics)

import json
rouge_path = OUTPUT_DIR / "rouge_scores_20pct.json"
with open(rouge_path, "w") as f:
    json.dump(metrics, f, indent=2)
print("Saved ROUGE to:", rouge_path)


In [ ]:
# 🔍 Quick qualitative check on a few samples
from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline("summarization", model=str(SAVE_DIR), tokenizer=str(SAVE_DIR), device=device)
sample_texts = df[TEXT_COL].head(3).tolist()
outs = summarizer(sample_texts, max_length=MAX_TARGET_LEN, truncation=True)
for i, o in enumerate(outs):
    print("=== SAMPLE", i+1, "===")
    print(o["summary_text"])

In [ ]:
# 📤 Generate summaries for the whole (20%) dataset and save CSV
from tqdm.auto import tqdm
preds = []
for i in tqdm(range(0, len(df), EVAL_BS)):
    batch = df[TEXT_COL].iloc[i:i+EVAL_BS].tolist()
    out = summarizer(batch, max_length=MAX_TARGET_LEN, truncation=True)
    preds.extend([x["summary_text"] for x in out])

out_df = df.copy()
out_df["generated_summary"] = preds
out_csv = OUTPUT_DIR / "summaries_finetuned_20pct.csv"
out_df.to_csv(out_csv, index=False)
print("Saved:", out_csv)
